In [4]:
from os import preadv
from gevent import config
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql.streaming import *
from pyspark.sql import SparkSession
import time
import pandas as pd
import config

In [5]:
spark = SparkSession.builder.appName("mnpl_data").getOrCreate()
df = spark.read.parquet("past_data")
df = df.toPandas()
df.head()

22/07/17 00:20:49 WARN Utils: Your hostname, Van.local resolves to a loopback address: 127.0.0.1; using 192.168.1.12 instead (on interface en0)
22/07/17 00:20:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/17 00:20:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
0,AAA,13.10,13.75,12.80,12.80,6651400.0,8.846755e+10,47,12.2775,13.680851,-8.332692,-10.325000,1.992308,20220610
1,AAM,12.20,12.65,12.10,12.40,14100.0,1.737000e+08,47,12.4350,13.540426,-8.580385,-10.558333,1.977949,20220610
2,ABT,41.25,41.25,40.15,41.25,3200.0,1.298400e+08,47,41.2825,40.873404,-28.688846,-35.282407,6.593561,20220610
3,ACB,25.20,25.80,25.15,25.20,2278300.0,5.790451e+10,47,28.2500,30.365957,-20.124231,-24.397222,4.272991,20220610
4,ACL,28.15,28.65,26.65,26.65,605300.0,1.662091e+10,47,26.1225,27.039362,-17.891538,-22.090741,4.199202,20220610


In [5]:
df.loc[df['Symbol'] == 'SJF'].sort_values(by="day", ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
2313,SJF,6.40,6.40,5.96,5.96,1009600.0,6.136374e+09,62,7.2665,9.8056,-5.415269,-6.487685,1.072416,20220701
5293,SJF,6.99,6.99,6.39,6.40,787000.0,5.238424e+09,61,7.4835,9.9304,-5.525269,-6.649537,1.124268,20220630
4399,SJF,7.08,7.08,6.45,6.75,1701700.0,1.172443e+10,60,7.6935,10.0644,-5.649115,-6.818056,1.168940,20220629
5591,SJF,6.23,6.62,6.22,6.62,277000.0,1.807974e+09,59,7.9035,10.2104,-5.850192,-7.025833,1.175641,20220628
7081,SJF,6.15,6.25,6.01,6.19,459500.0,2.830447e+09,58,8.1175,10.3800,-6.126192,-7.287315,1.161123,20220627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44986,SJF,15.95,16.50,15.95,16.00,1388100.0,2.244100e+10,5,16.4300,16.4300,-11.531731,-14.127315,2.595584,20220408
45541,SJF,16.80,16.80,16.20,16.20,1510600.0,2.492898e+10,4,16.5375,16.5375,-11.596154,-14.216667,2.620513,20220407
50169,SJF,16.60,17.30,16.50,16.75,2506100.0,4.233229e+10,3,16.6500,16.6500,-11.469231,-14.129630,2.660399,20220406
50442,SJF,16.45,16.85,16.40,16.60,1418400.0,2.361533e+10,2,16.6000,16.6000,-11.492308,-14.140741,2.648433,20220405


In [7]:
len(df.Symbol.unique())

866

In [6]:
mapping = df.loc[df['Symbol'] == 'SJF'][['day', 'PARTITION_DATE']]
day_mapping = {}
for ind, row in mapping.iterrows():
    key = row['PARTITION_DATE']
    day = row['day']
    day_mapping[key] = day



In [7]:
for i in range(len(df)):
    row = df.iloc[i]
    day = day_mapping[row['PARTITION_DATE']]
    df.loc[i, 'day'] = day

df.head()

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
0,AAA,13.10,13.75,12.80,12.80,6651400.0,8.846755e+10,47,12.2775,13.680851,-8.332692,-10.325000,1.992308,20220610
1,AAM,12.20,12.65,12.10,12.40,14100.0,1.737000e+08,47,12.4350,13.540426,-8.580385,-10.558333,1.977949,20220610
2,ABT,41.25,41.25,40.15,41.25,3200.0,1.298400e+08,47,41.2825,40.873404,-28.688846,-35.282407,6.593561,20220610
3,ACB,25.20,25.80,25.15,25.20,2278300.0,5.790451e+10,47,28.2500,30.365957,-20.124231,-24.397222,4.272991,20220610
4,ACL,28.15,28.65,26.65,26.65,605300.0,1.662091e+10,47,26.1225,27.039362,-17.891538,-22.090741,4.199202,20220610


In [24]:
len(df.loc[df['day'] == 62].Symbol.unique())

866

In [8]:
df.sort_values(by='day', ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
19350,EBS,10.0,10.2,10.00,10.00,1200.0,1.204000e+07,62,10.780,11.206,-7.629615,-9.291667,1.662051,20220701
21139,NLG,35.2,36.5,34.75,36.05,1826800.0,6.480834e+10,62,38.400,43.258,-27.288846,-33.260185,5.971339,20220701
21137,NHC,30.0,30.0,30.00,30.00,100.0,3.000000e+06,62,31.515,31.806,-22.135769,-27.050926,4.915157,20220701
21136,NHA,19.0,20.4,18.60,20.05,144600.0,2.770080e+09,62,22.895,29.670,-16.704808,-20.169907,3.465100,20220701
21135,NET,48.5,48.5,48.50,48.50,1100.0,5.335000e+07,62,48.935,51.538,-34.025385,-41.805556,7.780171,20220701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52132,YBM,11.0,11.2,10.90,10.95,65900.0,7.245250e+08,1,10.950,10.950,NaN,NaN,NaN,20220404
52133,ABR,20.4,20.6,20.05,20.60,600.0,1.224500e+07,1,20.600,20.600,NaN,NaN,NaN,20220404
52134,ACM,3.0,3.1,2.80,2.90,0.0,0.000000e+00,1,2.900,2.900,NaN,NaN,NaN,20220404
52135,ADG,51.8,51.8,51.00,51.60,170500.0,8.778700e+09,1,51.600,51.600,NaN,NaN,NaN,20220404


In [9]:
df['EMA12'] = df['Close']
df['EMA26'] = df['Close']
df

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
0,AAA,13.10,13.75,12.80,12.80,6651400.0,8.846755e+10,47,12.2775,13.680851,12.80,12.80,1.992308,20220610
1,AAM,12.20,12.65,12.10,12.40,14100.0,1.737000e+08,47,12.4350,13.540426,12.40,12.40,1.977949,20220610
2,ABT,41.25,41.25,40.15,41.25,3200.0,1.298400e+08,47,41.2825,40.873404,41.25,41.25,6.593561,20220610
3,ACB,25.20,25.80,25.15,25.20,2278300.0,5.790451e+10,47,28.2500,30.365957,25.20,25.20,4.272991,20220610
4,ACL,28.15,28.65,26.65,26.65,605300.0,1.662091e+10,47,26.1225,27.039362,26.65,26.65,4.199202,20220610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52666,VTO,15.20,15.50,15.00,15.00,1025500.0,1.552030e+10,1,15.0000,15.000000,15.00,15.00,NaN,20220404
52667,VTZ,13.30,13.40,13.10,13.30,12300.0,1.629500e+08,1,13.3000,13.300000,13.30,13.30,NaN,20220404
52668,WCS,190.00,190.00,190.00,190.00,0.0,0.000000e+00,1,190.0000,190.000000,190.00,190.00,NaN,20220404
52669,WSS,13.50,14.70,13.50,14.00,75100.0,1.055170e+09,1,14.0000,14.000000,14.00,14.00,NaN,20220404


In [10]:
k12 = 2/13
k26 = 2/27

res = pd.DataFrame(columns=df.columns)
for ticker in config.TICKERS:
    temp_df = df.loc[df["Symbol"] == ticker].copy()
    for i in range(61):
        if i == 0:
            temp_df.loc[temp_df['day'] == 1, 'MACD'] = 0
        if i + 1 not in df[df.Symbol == ticker]['day'].values:
            continue
        prev_ema12 = temp_df.loc[temp_df['day'] == i + 1, 'EMA12'].values[0]
        prev_ema26 = temp_df.loc[temp_df['day'] == i + 1, 'EMA26'].values[0]
        close = temp_df.loc[temp_df['day'] == i + 2, 'Close'].values[0]
        ema12 = close * k12 + prev_ema12 * (1-k12)
        ema26 = close * k26 + prev_ema26 * (1-k26)
        temp_df.loc[temp_df['day'] == i + 2, 'EMA12'] = ema12
        temp_df.loc[temp_df['day'] == i + 2, 'EMA26'] = ema26
        temp_df.loc[temp_df['day'] == i + 2, 'MACD'] = ema12 - ema26
    res = pd.concat([res, temp_df])
    
res.sort_values(by='day', ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
2214,HMH,16.8,17.4,16.8,17.4,5100.0,85740000.0,62,17.965,19.332,17.519721,18.301295,-0.781574,20220701
21026,CMX,16.8,17.3,16.25,16.95,1297100.0,21693295000.0,62,18.8425,19.04,17.640168,18.46673,-0.826562,20220701
19511,TPC,9.01,9.01,8.51,9.0,2500.0,21543000.0,62,9.3095,9.7134,9.26526,9.402153,-0.136893,20220701
21035,CTCB2201,0.06,0.07,0.05,0.05,611500.0,36500000.0,62,0.186,0.3404,0.135774,0.217877,-0.082102,20220701
20980,APC,18.0,18.5,17.35,18.45,3900.0,69085000.0,62,19.3125,20.534,18.757961,19.56796,-0.809999,20220701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52143,APC,26.4,26.7,26.1,26.6,27100.0,719020000.0,1,26.6,26.6,26.6,26.6,0.0,20220404
52476,FDC,37.0,38.0,36.9,37.0,103800.0,3864440000.0,1,37.0,37.0,37.0,37.0,0.0,20220404
52289,NKG,48.0,48.4,47.25,47.5,4664100.0,222192345000.0,1,47.5,47.5,47.5,47.5,0.0,20220404
52104,UNI,30.9,30.9,29.0,29.0,16300.0,480770000.0,1,29.0,29.0,29.0,29.0,0.0,20220404


In [13]:
res.loc[res["Symbol"] == 'FPT'].sort_values(by='day', ascending=False)

,Symbol,Open,High,Low,Close,Volume,Value,day,MA20,MA50,EMA12,EMA26,MACD,PARTITION_DATE
21079,FPT,87.0,88.8,84.8,88.0,1696500.0,146497780000.0,62,94.785,99.872,89.866139,94.49635,-4.630211,20220701
26755,FPT,89.5,89.9,86.2,86.2,1506500.0,132342490000.0,61,96.085,100.352,90.205437,95.016058,-4.810621,20220630
21647,FPT,88.9,90.0,88.2,90.0,1182600.0,105405850000.0,60,97.35,100.868,90.933698,95.721343,-4.787645,20220629
23638,FPT,89.5,90.2,87.2,89.7,2296600.0,204555720000.0,59,98.44,101.378,91.103461,96.17905,-5.075589,20220628
28175,FPT,86.4,87.9,85.9,86.5,1254800.0,109236740000.0,58,99.45,101.884,91.358636,96.697374,-5.338738,20220627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48706,FPT,113.3,113.9,108.5,108.5,4069700.0,450651660000.002991,5,110.62,110.62,110.077228,109.732423,0.344805,20220408
46529,FPT,113.7,116.9,112.1,113.3,3915200.0,450249650000.002014,4,111.15,111.15,110.363996,109.831017,0.53298,20220407
50881,FPT,108.4,114.0,108.4,113.0,4179000.0,465302340000.002991,3,110.433333,110.433333,109.830178,109.553498,0.27668,20220406
51152,FPT,109.1,111.6,109.0,109.0,1863900.0,205138060000.0,2,109.15,109.15,109.253846,109.277778,-0.023932,20220405


In [14]:
schema = (StructType()
            .add(StructField("Symbol", StringType()))
            .add(StructField("Open", StringType()))
            .add(StructField("High", StringType()))
            .add(StructField("Low", StringType()))
            .add(StructField("Close", StringType()))
            .add(StructField("Volume", StringType()))
            .add(StructField("Value", StringType()))
            .add(StructField("day", StringType()))
            .add(StructField("MA20", StringType()))
            .add(StructField("MA50", StringType()))
            .add(StructField("EMA12", StringType()))
            .add(StructField("EMA26", StringType()))
            .add(StructField("MACD", StringType()))
            .add(StructField("PARTITION_DATE", StringType()))    
        )
spark_df = spark.createDataFrame(res, schema=schema)


In [17]:
!export PYSPARK_PYTHON="/usr/local/opt/python@3.8/bin/python3"
!export PYSPARK_DRIVER_PYTHON="/usr/local/opt/python@3.8/bin/python3"

In [18]:
spark_df.show()

22/07/17 00:30:25 WARN TaskSetManager: Stage 4 contains a task of very large size (1485 KiB). The maximum recommended task size is 1000 KiB.
22/07/17 00:30:25 ERROR Executor: Exception in task 0.0 in stage 4.0 (TID 70)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 477, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.8, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:503)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:638)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePyth

Py4JJavaError: An error occurred while calling o53.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 70, 192.168.1.12, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 477, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.8, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:503)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:638)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:621)
	a

lExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:834)

22/07/17 00:30:25 ERROR TaskSetManager: Task 0 in stage 4.0 failed 1 times; aborting job
